In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import re
import string
from nltk.tokenize import RegexpTokenizer

In [43]:
def load_data(text_file):
    with open(text_file, 'r') as f:
        lines = f.readlines()

    lines = [line.strip() for line in lines]
    english_text = []
    telugu_text = []
    
    for line in lines:
        parts = line.split('++++$++++')
        english_text.append(parts[0].strip())
        telugu_text.append(parts[1].strip())
    
    return english_text, telugu_text

path = 'english_telugu_data.txt'
english_sentences, telugu_sentences = load_data(path)
print(len(english_sentences))
print(len(telugu_sentences))

155798
155798


In [44]:
english_sentences = english_sentences[:30000]
len(english_sentences)

30000

In [45]:
telugu_sentences = telugu_sentences[0:30000]
len(telugu_sentences)

30000

In [64]:
def tokenize(sentences):
    vocab = set()
    tokenized_sentences = []
    
    for sentence in sentences:
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        words = sentence.split()
        tokenized_sentences.append(words)
        for word in words:
            vocab.add(word)
    return tokenized_sentences,list(vocab)

english_sentences_tokenized, english_vocab = tokenize(english_sentences)
telugu_sentences_tokenized, telugu_vocab = tokenize(telugu_sentences)

print(f' len(english_vocab): {len(english_vocab)}')
print(f' len(telugu_vocab): {len(telugu_vocab)}')

for i, vocab in enumerate(telugu_vocab):
    vocab = vocab.replace('\u200c','')
    telugu_vocab[i] = vocab

telugu_vocab  += ['<pad>','<unk>','<sos>','<eos>']
english_vocab += ['<pad>','<unk>','<sos>','<eos>']

 len(english_vocab): 8137
 len(telugu_vocab): 17400


In [70]:
english_itow = {i: w for i, w in enumerate(english_vocab)}
english_wtoi = {w:i for i, w in english_itow.items()}

telugu_itow = {i: w for i, w in enumerate(telugu_vocab)}
telugu_wtoi = {w:i for i, w in telugu_itow.items()}